### Load data

- ST: SESSION --> START, TIIME_DELTA
- SRM: ROOM, SESSION --> TYPE
- SRP: SESSION, ROOM, S_ORDER --> FIRST_NAME, LAST_NAME

To update SRP, use [SRP collector](#SRP-collector).  
To update srp_full, use [Merge SRP with talks](#Merge-SRP-with-talks).

In [ ]:
from urllib.request import urlretrieve

URLS = {
    "minis.csv": "https://docs.google.com/spreadsheets/d/1IQkcuu7cRXA5wlwMJspwSSfZNHIdXoapZBKufdFIayc/export?format=csv&gid=0", 
    "ST.csv": "https://docs.google.com/spreadsheets/d/1IQkcuu7cRXA5wlwMJspwSSfZNHIdXoapZBKufdFIayc/export?format=csv&gid=1165578014", 
    "SRM.csv": "https://docs.google.com/spreadsheets/d/1IQkcuu7cRXA5wlwMJspwSSfZNHIdXoapZBKufdFIayc/export?format=csv&gid=244777042", 
    "SRP.csv": "https://docs.google.com/spreadsheets/d/1IQkcuu7cRXA5wlwMJspwSSfZNHIdXoapZBKufdFIayc/export?format=csv&gid=1456705243" 
}

for filename, url in URLS.items():
    urlretrieve(url, filename)

In [ ]:
# global DataFrame
import re
import pandas as pd

minis = pd.read_csv("minis.csv").set_index("MID")
minis.loc["TMS"] = ["Test mini-symposium", "Test organizers"]
minis.loc["CT"] = ["Contributed talks", ""]

SESSION_NAMES = {
    "Reg": "Registration", 
    "OR": "Opening Remarks", 
    "Rec": "Reception", 
    "Photo": "Photo", 
    "BM": "Business Meeting", 
    "E": "Excursion", 
    "Ban": "Banquet", 
    "CR": "Closing Remarks"
}
for s,num,name in [("P",10,"Plenary Talk"), ("CB",8,"Coffee Break"), ("S",11,"Mini-symposium"), ("LB",4,"Lunch Break")]:
    for t in range(1, num + 1):
        SESSION_NAMES[s + "%d"%t] = name
SESSION_NAMES["S3"] = "Contributed Session"
SESSION_NAMES["S11"] = "Contributed Session"

SESSION_VENUES = {
    "Reg": "SYS Hall", 
    "OR": "SYS Hall", 
    "Rec": "College of Science", 
    "Photo": "SYS Hall", 
    "BM": "SYS Hall", 
    "E": "Meet at SYS Hall", 
    "Ban": "College of Science", 
    "CR": "SYS Hall"
}
for s,num,name in [("P",10,"SYS Hall"), ("CB",8,"College of Science"), ("S",11,"College of Science"), ("LB",4,"")]:
    for t in range(1, num + 1):
        SESSION_VENUES[s + "%d"%t] = name

def session_weight(name):
    if pd.isna(name):
        return 1000
    if re.match("P[0-9]{1,2}$", name):
        return int(name[1:])
    if re.match("S[0-9]{1,2}$", name):
        return 100 + int(name[1:])

def type_weight(name):
    if pd.isna(name):
        return 1000
    if re.match("MS[0-9]{1,2}$", name):
        return int(name[2:])
    if name == "Plenary":
        return 100
    if name == "CT":
        return 200    

st = pd.read_csv("ST.csv")
st["START"] = pd.to_datetime(st.START, format="%Y-%m-%d %H:%M")
st["TIME_DELTA"] = pd.to_timedelta(st.TIME_DELTA, unit="m")
st.insert(2, "END", st.START + st.TIME_DELTA)
st["SESSION_NAME"] = st.SESSION.apply(lambda x: SESSION_NAMES[x])
st["SESSION_VENUE"] = st.SESSION.apply(lambda x: SESSION_VENUES[x])
st["START_STR"] = st.START.dt.strftime("%H:%M")
st["END_STR"] = st.END.dt.strftime("%H:%M")
st.set_index("SESSION", inplace=True)

srm = pd.read_csv("SRM.csv", header=1).set_index("ROOM")

srp = pd.read_csv("SRP.csv")
srp["FULL_NAME"] = srp.FIRST_NAME + " " + srp.LAST_NAME
srp = srp.join(st.loc[:,["START","START_STR","END_STR"]], on="SESSION")
srp.insert(7, "WEEKDAY", srp.START.dt.strftime("%A"))
p_mask = srp.SESSION.str.match("P[0-9]{1,2}$")
ms_mask = srp.SESSION.str.match("S[0-9]{1,2}$")
srp.loc[ms_mask,"START_STR"] = (srp.loc[ms_mask,"START"] + pd.to_timedelta(30 * (srp.loc[ms_mask,"S_ORDER"] - 1), "m")).dt.strftime("%H:%M")
srp.loc[ms_mask,"END_STR"] = (srp.loc[ms_mask,"START"] + pd.to_timedelta(30 * srp.loc[ms_mask,"S_ORDER"], "m")).dt.strftime("%H:%M")
srp.insert(7, "TYPE", None)
srp.insert(8, "MS_TITLE", None)
srp.loc[p_mask,"TYPE"] = "Plenary"
srp.loc[ms_mask,"TYPE"] = srp.loc[ms_mask].apply(lambda x: srm.loc[x.ROOM, x.SESSION], axis=1)
srp.loc[ms_mask,"MS_TITLE"] = srp.loc[ms_mask].apply(lambda x: minis.TITLE[x.TYPE] if pd.notna(x.TYPE) else x.TYPE, axis=1)
srp["SESSION_ORDER"] = srp.SESSION.apply(session_weight)
srp["TYPE_ORDER"] = srp.TYPE.apply(type_weight)
srp = srp.set_index(["SESSION","ROOM","S_ORDER"])

srp_full = pd.read_csv("srp-full.csv").set_index(["SESSION","ROOM","S_ORDER"])
srp_full["START"] = pd.to_datetime(srp_full.START)
num_talk = srp_full.shape[0]
srp_full["DOWN_LABEL"] = ["down%04d"%i for i in range(num_talk)]
srp_full["UP_LABEL"] = ["up%04d"%i for i in range(num_talk)]

In [ ]:
# minis.head()
# st.head()
srp.head()
# srp.loc[ms_mask].head()
# srm.head()
# srp_full.head()

### SRP collector

In [ ]:
# SRP --> SRP-collector
collector_columns = ["TYPE","MS_TITLE","WEEKDAY","START_STR","END_STR","FIRST_NAME","LAST_NAME"]

srp.sort_values(["TYPE_ORDER","START"]).loc[:,collector_columns].to_csv("srp-collector.csv")

In [ ]:
# SRP-collector --> SRP
srp_get = pd.read_csv("srp-get.csv")
srp_get["SESSION_ORDER"] = srp_get.SESSION.apply(session_weight)
srp_columns = ["SESSION", "ROOM", "S_ORDER", "FIRST_NAME", "LAST_NAME"]
srp_get.sort_values(["SESSION_ORDER", "ROOM", "S_ORDER"]).loc[:,srp_columns].to_csv("SRP.csv", index=False)

### Merge SRP with talks

get global DataFrame first

In [ ]:
# Merge SRP with talks
def type_abbreviate(name):
    if name == "Plenary talk":
        return "Plenary"
    elif name == "Contributed talk":
        return "CT"
    elif re.match("MS[0-9]{1,2}:", name):
        return name.split(":")[0]
    else:
        raise ValueError("Unexpected input")

talks_columns = ["FIRST_NAME", "LAST_NAME", "TYPE", "TITLE", "ABSTRACT"]
talks = pd.read_csv("talks.csv").set_index("TID").loc[:,talks_columns].dropna(how="all")
talks["TYPE"] = talks.TYPE.apply(type_abbreviate)
# talks.head()

In [ ]:
# validate every talk row merge with some srp row
srp_full = srp.merge(talks, on=["TYPE", "FIRST_NAME", "LAST_NAME"], how="outer", indicator=True)
# error_mask = (srp_full._merge == "right_only")
error_mask = (srp_full._merge == "right_only") & (srp_full.TYPE != "CT")
srp_full.loc[error_mask]

In [ ]:
srp_full = srp.reset_index().merge(talks, on=["TYPE", "FIRST_NAME", "LAST_NAME"], how="left")
srp_full = srp_full.set_index(["SESSION","ROOM","S_ORDER"])
print(srp.shape[0], srp_full.shape[0])
srp_full.head()

In [ ]:
srp_full.to_csv("srp-full.csv")

### SRP creator

In [ ]:
# independent of the setting from ST
ind = pd.MultiIndex.from_product([
    ["S%d"%i for i in range(1, 12)], 
    ["SC0008", "SC0009", "SC0012", "SC0014", "SC1001", "SC1003", "SC1005", "SC2001", "SC2006", "SC3001", "SC4011"]
], names=["SESSION", "ROOM"])

ss = ["S%d"%i for i in range(1, 12)]
rs = ["SC0008", "SC0009", "SC0012", "SC0014", "SC1001", "SC1003", "SC1005", "SC2001", "SC2006", "SC3001", "SC4011"]
s_num = [3,3,4,3,4,4,3,3,4,4,3]
tuples = []
for i in range(11):
    for r in rs:
        for j in range(1, s_num[i]+1):
            tuples.append((ss[i], r, j))

ind = pd.MultiIndex.from_tuples(tuples)
df = pd.DataFrame({"key": 1}, index=ind)

df.to_csv("temp.csv")

### Required functions

In [ ]:
import re
import os
import subprocess
import pandas as pd

def cjken(s, prefix=""):
    """
    Input:
        s: string
    Output:
        enclose Chinese with \CJKEN{...}
    """
    if s != "":
        return prefix + r"\CJKEN{%s}"%s
    else:
        return ""

def write_newpage():
    f.write(r"""\newpage"""+'\n\n')
    
def write_sr(s_ind, r_ind):
    m_ind = srm.loc[r_ind, s_ind]
    if pd.isna(m_ind):
        return None
    presentations = srp_full.loc[s_ind, r_ind]
    f.write(r"""\begin{description}
    \item[] {\color{mstitle}\textbf{%s}} \info{%s}
    """%(m_ind + ": " + minis.loc[m_ind,"TITLE"], r_ind))

    # write each talk in each session
    for s_order, row in presentations.iterrows():
        f.write(r"""\item[\info{%s\textrm{--}%s}] \hypertarget{%s}{}\textbf{%s}, \hyperlink{%s}{%s}
        """%(row.START_STR, 
             row.END_STR, 
             row.UP_LABEL, 
             row.FULL_NAME, 
             row.DOWN_LABEL, 
             row.TITLE))

    f.write(r"""\end{description}
    """)

def write_a_session(s_ind):
    # write session time and title
    row = st.loc[s_ind]
    f.write(r"""\item[\info{%s\textrm{--}%s}] \textbf{%s} \info{%s}
    """%(row.START_STR, row.END_STR, row.SESSION_NAME, row.SESSION_VENUE))

    # write extra information for plenary sessions # , %s , presentation.TITLE
    if re.match("P[0-9]{1,2}$", s_ind):
        presentation = srp_full.loc[s_ind].iloc[0]
        f.write(r"""\begin{description}
        \item[] \hypertarget{%s}{}\textbf{%s}, \hyperlink{%s}{%s}
        \end{description}
        """%(presentation.UP_LABEL, 
             presentation.FULL_NAME, 
             presentation.DOWN_LABEL, 
             presentation.TITLE))

    # write extra information for parallel sessions
    if re.match("S[0-9]{1,2}$", s_ind):
        s_all_room = srp_full.loc[s_ind]
        for r_ind in srp_full.loc[s_ind].index.get_level_values(0).unique():
            write_sr(s_ind, r_ind)

def write_a_talk(s_ind, r_ind, s_ord):
    talk = srp_full.loc[s_ind, r_ind, s_ord]
    if pd.isna(talk.FULL_NAME):
        return None
    if talk.TYPE.startswith("MS"):
        m_ind = srm.loc[r_ind, s_ind]
    f.write(r"""

\hypertarget{%s}{}\begin{ilasabstract}
\talktitle{%s}
    
\textbf{%s}, \info{%s} \hfill \hyperlink{%s}{$\Uparrow$}
    
    """%(talk.DOWN_LABEL, 
         talk.TITLE, 
         talk.FULL_NAME, 
         talk.START_STR + r"\textrm{--}" + talk.END_STR + " @ " + r_ind + talk.START.strftime(" (%B %-d, %A)"),
         talk.UP_LABEL
        ))
    
    if talk.TYPE.startswith("MS"):
        f.write(r"""
(in {\color{mstitle}%s})
        """%(m_ind + ": " + minis.loc[m_ind,"TITLE"]))

    f.write(r"""
\mtskip
    """)

    if re.search("bibitem", "%s"%talk.ABSTRACT):
        f.write(r"""\begin{bibunit}
        %s
        \end{bibunit}
        """%talk.ABSTRACT)
    else:
        f.write("%s"%talk.ABSTRACT)

    f.write(r"""
\end{ilasabstract}
    """)

### Schedule

In [ ]:
##### CREATE SCHEDULE.TEX #####
import warnings
warnings.filterwarnings("ignore", message="indexing past lexsort depth may impact performance")

# warnings.warn("General warnings still occur")

# beginning of schedule.tex
f = open('schedule.tex','w')

f.write(r"""\documentclass[ILAS2025-program.tex]{subfiles}

\begin{document}

""")

# write five day schedule
days = pd.date_range("2025-06-23", "2025-06-27")

for day in days:
    f.write(r"""\section{%s}
    
    \begin{description}
    """%day.strftime("%A Schedule, %B %d"))
    
    today = st.index[st.START.dt.strftime("%d") == day.strftime("%d")]

    for s_ind in today:
        write_a_session(s_ind)
    
    f.write(r"""\end{description}
    """)
    
    write_newpage()

f.write(r"""\end{document}
""")

f.close()
# ending of schedule.tex

# beginning of abstract.tex
f = open('abstract.tex','w')

f.write(r"""\documentclass[ILAS2025-program.tex]{subfiles}

\begin{document}

\parindent=0pt

""")

f.write(r"""\section{Abstracts of Plenary Talks}
""")

p_talks = srp_full.loc[srp_full.TYPE == "Plenary"].sort_values(["LAST_NAME", "SESSION_ORDER"])
for ind, row in p_talks.iterrows():
    write_a_talk(*ind)

write_newpage()

f.write(r"""\section{Abstracts of Mini-symposium Talks}
""")

ms_talks = srp_full.loc[srp_full.TYPE.str.startswith("MS", False)].sort_values(["LAST_NAME", "SESSION_ORDER"])
for ind, row in ms_talks.iloc[:].iterrows():
    write_a_talk(*ind)

write_newpage()

f.write(r"""\section{Abstracts of Contributed Talks}
""")

ct_talks = srp_full.loc[srp_full.TYPE == "CT"].sort_values(["LAST_NAME", "SESSION_ORDER"])
for ind, row in ct_talks.iterrows():
    write_a_talk(*ind)

write_newpage()

f.write(r"""\end{document}
""")

f.close()
# ending of abstract.tex

subprocess.run(["pdflatex", "-interaction=batchmode", "ILAS2025-program.tex"])

### Archived

In [ ]:
def write_mtskip():
    f.write(r"""\mtskip"""+'\n\n')
    
def write_btskip():
    f.write(r"""\btskip"""+'\n\n')

def write_a_talk(file, data, ind, email=True, advisor=False):
    file.write(r'''{\parindent=0pt'''+'\n\n')
    file.write(r'''\hypertarget{'''+ind+r'''}{\talktitle{'''+data.loc[ind,'title']+'}}\n\n\\nopagebreak{}')
    file.write(r'''\centerline{%s, %s \hfill \hyperlink{%s}{$\Uparrow$}}'''%(
        data.loc[ind,'name'],data.loc[ind,'affiliation'],ind+'top')+'\n\n')
    if email:
        file.write(", ".join(r"""\url{%s}"""%addrs for addrs in data.loc[ind,'email'].split(",")) + '\n\n')
    if advisor:
        file.write(r"""Advisor: %s"""%data.loc[ind,'advisor'] + '\n\n')
    file.write('}\n\n\\nopagebreak{}')
    file.write(r'''\vspace{0.5cm}'''+'\n\n\\nopagebreak{}')
    file.write(data.loc[ind,'abstract']+'\n\n')
    file.write(r'''\btskip'''+'\n\n')
    
def write_name_title(file, data, advisor=False):
    file.write(r'''{\parindent=0pt'''+'\n\n')
    for ind in data.index:
        file.write(r'''\hypertarget{%s}\centerline{%s, %s \hfill \hyperlink{%s}{$\Uparrow$}}'''%(
            ind, 
            data.loc[ind,'name'], 
            data.loc[ind,'title'], 
            ind+'top')+'\n\n')
        file.write(r"""Advisor: %s"""%data.loc[ind,'advisor'] + '\n\n')
    file.write('}')